In [1]:
# author:kune806
# date:2018/12/5


import pandas as pd
import numpy as np
import os


filepath = "WIOT2014_Nov16_ROW.csv"
temporyurl = "rh12051226.csv"  # 仅作为数据变量类型的重新识别，无它用，不用更改
_spcountry = "CHN"  # 分析的国家，国家三位码，大写，简写。确保在投入产出表中存在

In [2]:
initialdb = pd.read_csv(filepath, encoding='gb18030', low_memory=False)

In [3]:
# rename columns of the initial IO table
initialdb = initialdb.iloc[1:,:]
country = initialdb.iloc[2, :].values
industry = initialdb.iloc[0, :].values
colname = ["".join([str(corei), "_", str(corej)]) for corei, corej in zip(country, industry)]
keys = initialdb.keys()
recolumns = dict()
for dici, dicj in zip(keys[4:], colname[4:]):
    recolumns[dici] = dicj
initialdb = initialdb.rename(columns=recolumns)

In [4]:
initialdb.keys()
initialdb = initialdb.iloc[4:, :]
fname = initialdb.rename(columns={"Intercountry Input-Output Table": "indcode", "Unnamed: 1": "des", "Unnamed: 2": "country",
                                     "Unnamed: 3": "order"})
fname.to_csv(temporyurl, encoding='gb18030', index=False)
fname = pd.read_csv(temporyurl, encoding='gb18030', low_memory=False)
os.remove(temporyurl)

In [5]:
# 中间投入产出矩阵, 行向
imi = fname[fname.country != 'TOT']

# 剩余数据表，行向
imi_res = fname[fname.country == 'TOT']

# 国家集合lcountry, 产品集合lproduct
lcountry = set(fname["country"].values) - set({"TOT"})
lproduct = set(imi["indcode"].values)

# 关注的国家和产品
spcountry = {_spcountry}
att_ind = ['A01', 'A02', 'A03', 'C10-C12', 'C13-C15', 'C16', 'C17', 'C31_C32']

# 无关国家和产品
rescountry = lcountry - spcountry
resproduct = lproduct - set(att_ind)

# 最终消费
finald = ["CONS_h", "CONS_np", "CONS_g", "GFCF", "INVEN"]

In [6]:
# special a country and special industry range, row combine
def rowind(analrow, attrind, countryvar):
    """
    analyrow is database, attrind is vector of key industry, countryvar is file name
    function: extract, merge 
    """
    logical_l = [stri in attrind for stri in analrow["indcode"].values]
    logical_f = [not(stri in attrind) for stri in analrow["indcode"].values]
    rowdb = analrow[logical_l]
    
    rowdb_res = analrow[logical_f]
    sumdb = np.float_(rowdb_res.iloc[:, 4:].values)
    sumr = np.array(np.sum(sumdb, axis=0))
    frontstr = np.array(["resind", "other", countryvar, "other"])
    sumrc = np.r_["0,1", frontstr, sumr]
    sumrcdf = pd.DataFrame(np.array([sumrc]), columns=list(analrow.keys()))
    return pd.concat([rowdb, sumrcdf])


# residual country 
def resrow(analrow, attrind):
    logical_l = [stri in attrind for stri in analrow["indcode"].values]  # eatrct special industries
    logical_f = [not(stri in attrind) for stri in analrow["indcode"].values]   # residual industries
    analrow.loc[:, "country"] = ["resc"] * analrow.shape[0]
    analrow.loc[:, "order"] = [1] * analrow.shape[0]
    
    resdbf = analrow[logical_f]
    spedbt = analrow[logical_l]
    
    resdbf.loc[:, "indcode"] = ["resind"] * resdbf.shape[0]
    resdbf.loc[:, "des"] = ["other industries"] * resdbf.shape[0]
    
    anal_combine = pd.concat([resdbf, spedbt], sort=False)
    
    anal_group = anal_combine.groupby(by=["indcode", "des", "country", "order"]).sum()
    anal_reset = anal_group.reset_index()
    
    return anal_reset


# 无关产业合并函数,列合并
def colcombine(analydb, countrylist, productlist, varname):
    columns = ["".join([countryi, "_", producti]) for countryi in countrylist for producti in productlist]
    analydb_sub = analydb.reindex(columns=columns)
    sum_temp = np.sum(np.float_(analydb_sub.values), axis=1)
    return pd.DataFrame(sum_temp, columns=[varname])


# 关注产业合并函数，列合并
def colcombine_c(analydb, countrylist, productlist, varname):
    result_list = list()
    for elementp in productlist:
        columns = ["".join([countryi, "_", elementp]) for countryi in countrylist]
        analydb_sub = analydb.reindex(columns=columns)
        sum_temp = np.sum(np.float_(analydb_sub.values), axis=1)
        eledf = pd.DataFrame(sum_temp, columns=["".join([varname, "_", elementp])])
        
        result_list.append(eledf)
    return pd.concat(result_list, axis=1, sort=True)

In [7]:
chn = rowind(imi[imi.country == _spcountry], att_ind, _spcountry)
resc = resrow(imi[imi.country != _spcountry], att_ind)

d:\pythonins\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [8]:
row_final = pd.concat([chn, resc, imi_res], sort=True)

In [9]:
CHN_col = colcombine(row_final, [_spcountry], resproduct, "%s_resind" % _spcountry)
rescp_col = colcombine(row_final, rescountry, resproduct, "resc_resind")

In [10]:
resc_ap_col = colcombine_c(row_final, rescountry, att_ind, "resc")
CHN_c_col = colcombine_c(row_final, [_spcountry], att_ind, _spcountry)

CHN_f = colcombine(row_final, [_spcountry], finald, "%s_demand" % _spcountry)
rescp_f = colcombine(row_final, rescountry, finald, "resc_demand")

In [11]:
namedb = row_final.reindex(columns=["indcode", "des", "country", "order"])
namedb.index = range(namedb.shape[0])
outdb = row_final.reindex(columns=["TOT_GO"])
outdb.index = range(outdb.shape[0])
finalresult = pd.concat([namedb, CHN_c_col, CHN_col, resc_ap_col, rescp_col, CHN_f, rescp_f, outdb], axis=1, sort=True)
finalresult.to_csv("rh12051431.csv", encoding='gb18030', index=False)